# データセットの使用

前のラボでは、*datastore*を使用して、クラウドベースの集中データアクセスを提供していました。このラボでは、*datasets*を探索します。これは、実験とトレーニングのために特定のデータを簡単に操作できるようにするさらなる抽象化です。

## ワークスペースに接続する

最初に行う必要があるのは、Azure ML SDKを使用してワークスペースに接続することです。

> **Note**: 前の演習を完了してから、Azureサブスクリプションとの認証済みセッションの有効期限が切れた場合、再認証するように求められます。

In [ ]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

## データを準備する

前のラボでは、データストアを作成しました。データセットは通常（常にではありませんが）データストア内のデータに基づいています。

前のラボを完了していない場合は、次のコードを実行して、2つのローカルCSVファイルをワークスペースのデフォルトのデータストアにアップロードします（前のラボを*完了*した場合、同じファイルが上書きされます）。

In [ ]:
ws.get_default_datastore().upload_files(files=['./data/diabetes.csv', './data/diabetes2.csv'], # Upload the diabetes csv files in /data
                       target_path='diabetes-data/', # Put it in a folder path in the datastore
                       overwrite=True, # Replace existing files of the same name
                       show_progress=True)

## 表形式のデータセットを作成する

データセットは、特定のデータソースをカプセル化するオブジェクトです。データストアにアップロードした糖尿病データからデータセットを作成し、最初の20レコードを表示してみましょう。この場合、データはCSVファイルの構造化された形式であるため、*Tabular*データセットを使用します。

In [ ]:
from azureml.core import Dataset

# Get the default datastore
default_ds = ws.get_default_datastore()

#Create a tabular dataset from the path on the datastore (this may take a short while)
tab_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'diabetes-data/*.csv'))

# Display the first 20 rows as a Pandas dataframe
tab_data_set.take(20).to_pandas_dataframe()

上記のコードを見るとわかるように、表形式のデータセットをPandasデータフレームに簡単に変換でき、一般的なpythonテクニックを使用してデータを操作できます。

## ファイルデータセットを作成する

作成したデータセットは、表形式のデータセットであり、データセット定義に含まれる構造化ファイルのすべてのデータを含むデータフレームとして読み取ることができます。これは表形式のデータではうまく機能しますが、一部の機械学習シナリオでは、構造化されていないデータを使用する必要がある場合があります。または、独自のコード内のファイルからデータの読み取りを処理することもできます。これを実現するには、*file*データセットを使用します。これにより、仮想マウントポイントにファイルパスのリストが作成され、ファイル内のデータを読み取ることができます。

In [ ]:
#Create a file dataset from the path on the datastore (this may take a short while)
file_data_set = Dataset.File.from_files(path=(default_ds, 'diabetes-data/*.csv'))

# Get the files in the dataset
for file_path in file_data_set.to_path():
    print(file_path)

## データセットの登録

糖尿病データを参照するデータセットを作成したので、それらを登録して、ワークスペースで実行されている実験に簡単にアクセスできるようにすることができます。

テーブルデータセットを**diabetes datasets**として、ファイルデータセットを**diabetes file**として登録します。

In [ ]:
# Register the tabular dataset
tab_data_set = tab_data_set.register(workspace=ws, 
                           name='diabetes dataset',
                           description='diabetes data',
                           tags = {'format':'CSV'},
                           create_new_version=True)

# Register the file dataset
file_data_set = file_data_set.register(workspace=ws, 
                           name='diabetes file dataset',
                           description='diabetes files',
                           tags = {'format':'CSV'},
                           create_new_version=True)

print('Datasets registered')

[Azure ML Studio](https://ml.azure.com)のワークスペースの**Datasets**ページでデータセットを表示および管理できます。また、ワークスペースオブジェクトからデータセットのリストを取得できます。:

In [ ]:
print("Datasets:")
for dataset_name in list(ws.datasets.keys()):
    dataset = Dataset.get_by_name(ws, dataset_name)
    print("\t", dataset.name, 'version', dataset.version)

Labs 2Aおよび2Bを完了すると、登録されたデータセットにビジュアルデザイナーツールを使用して作成された変換が含まれていることがわかります。また、前の演習で*Studio*インターフェイスを使用して作成したデータセットと同じ名前で**diabetes datasets**を登録すると、データセットの新しい*バージョン*が作成されることに気付くかもしれません。データセットをバージョン管理する機能により、以前の定義に依存する既存の実験またはパイプラインを壊すことなく、データセットを再定義できます。デフォルトでは、名前付きデータセットの最新バージョンが返されますが、次のようにバージョン番号を指定することで、データセットの特定のバージョンを取得できます:

```python
dataset_v1 = Dataset.get_by_name(ws, 'diabetes dataset', version = 1)
```


## 表形式のデータセットからモデルをトレーニングする

データセットができたので、データセットからトレーニングモデルを開始する準備ができました。データセットを、スクリプトの実行に使用されている推定器の*inputs*としてスクリプトに渡すことができます。

次の2つのコードセルを実行して作成します:

1. **diabetes_training_from_tab_dataset**という名前のフォルダー
2. 渡される表形式のデータセットを使用して分類モデルをトレーニングするスクリプトは、*input*としてです。

In [ ]:
import os

# Create a folder for the experiment files
experiment_folder = 'diabetes_training_from_tab_dataset'
os.makedirs(experiment_folder, exist_ok=True)
print(experiment_folder, 'folder created')

In [ ]:
%%writefile $experiment_folder/diabetes_training.py
# Import libraries
import argparse
from azureml.core import Run
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# Set regularization hyperparameter (passed as an argument to the script)
parser = argparse.ArgumentParser()
parser.add_argument('--regularization', type=float, dest='reg_rate', default=0.01, help='regularization rate')
args = parser.parse_args()
reg = args.reg_rate

# Get the experiment run context
run = Run.get_context()

# load the diabetes data (passed as an input dataset)
print("Loading Data...")
diabetes = run.input_datasets['diabetes'].to_pandas_dataframe()

# Separate features and labels
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Train a logistic regression model
print('Training a logistic regression model with regularization rate of', reg)
run.log('Regularization Rate',  np.float(reg))
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

os.makedirs('outputs', exist_ok=True)
# note file saved in the outputs folder is automatically uploaded into experiment record
joblib.dump(value=model, filename='outputs/diabetes_model.pkl')

run.complete()

これで、スクリプトを実行する推定器を作成し、スクリプトによって読み取られるトレーニングデータセットの名前付き*input*を定義できます。

> **Note**: **Dataset**クラスは**azureml-dataprep**パッケージ（SDKと共にインストールされます）で定義され、このパッケージには**pandas**(**to_pandas_dataframe()**)メソッド。そのため、トレーニングパッケージを実行する環境にこのパッケージを含める必要があります。

In [ ]:
from azureml.train.sklearn import SKLearn
from azureml.core import Experiment
from azureml.widgets import RunDetails

# Set the script parameters
script_params = {
    '--regularization': 0.1
}

# Get the training dataset
diabetes_ds = ws.datasets.get("diabetes dataset")

# Create an estimator
estimator = SKLearn(source_directory=experiment_folder,
                    entry_script='diabetes_training.py',
                    script_params=script_params,
                    compute_target = 'local',
                    inputs=[diabetes_ds.as_named_input('diabetes')], # Pass the Dataset object as an input...
                    conda_packages=['pip==19.3.1'],
                    pip_packages=['azureml-dataprep[pandas]'] # ...so you need the dataprep package
                   )

# Create an experiment
experiment_name = 'diabetes-training'
experiment = Experiment(workspace = ws, name = experiment_name)

# Run the experiment
run = experiment.submit(config=estimator)
# Show the run details while running
RunDetails(run).show()
run.wait_for_completion()

実験を初めて実行するときは、Python環境のセットアップに時間がかかる場合があります-以降の実行はより高速になります。

実験が完了したら、ウィジェットで**azureml-logs/70_driver_log.txt**出力ログと実行によって生成されたメトリックを表示します。

すべての実験と同様に、[Azure ML Studio](https://ml.azure.com)で実験の詳細を表示でき、生成されたメトリックとファイルを取得するコードを作成できます。:

In [ ]:
# Get logged metrics
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')
for file in run.get_file_names():
    print(file)

トレーニングしたモデルは**diabetes_model.pkl**ファイルとして**outputs**フォルダーに保存されるため、登録できます。

In [ ]:
from azureml.core import Model

run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                   tags={'Training context':'SKLearn Estimator (tabular dataset)'}, properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

## ファイルデータセットからモデルをトレーニングする

*表*データセットのトレーニングデータを使用してモデルをトレーニングする方法を見てきました。しかし、*file*データセットはどうですか？

ファイルデータセットを使用している場合、スクリプトに渡されるデータセット入力は、ファイルパスを含むマウントポイントを表します。これらのファイルからデータを読み取る方法は、ファイル内のデータの種類とそれをどうするかによって異なります。糖尿病CSVファイルの場合、Python **glob**モジュールを使用して、データセットによって定義された仮想マウントポイントにファイルのリストを作成し、それらをすべて単一のデータフレームに連結されたPandasデータフレームに読み込むことができます。

次の2つのコードセルを実行して作成します:

1. **diabetes_training_from_file_dataset**という名前のフォルダー
2. 渡されるファイルデータセットを使用して分類モデルをトレーニングするスクリプトは、*input*です。

In [ ]:
import os

# Create a folder for the experiment files
experiment_folder = 'diabetes_training_from_file_dataset'
os.makedirs(experiment_folder, exist_ok=True)
print(experiment_folder, 'folder created')

In [ ]:
%%writefile $experiment_folder/diabetes_training.py
# Import libraries
import argparse
from azureml.core import Workspace, Dataset, Experiment, Run
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import glob

# Set regularization hyperparameter (passed as an argument to the script)
parser = argparse.ArgumentParser()
parser.add_argument('--regularization', type=float, dest='reg_rate', default=0.01, help='regularization rate')
args = parser.parse_args()
reg = args.reg_rate

# Get the experiment run context
run = Run.get_context()

# load the diabetes dataset
print("Loading Data...")
data_path = run.input_datasets['diabetes'] # Get the training data from the estimator input
all_files = glob.glob(data_path + "/*.csv")
diabetes = pd.concat((pd.read_csv(f) for f in all_files))

# Separate features and labels
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Train a logistic regression model
print('Training a logistic regression model with regularization rate of', reg)
run.log('Regularization Rate',  np.float(reg))
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

os.makedirs('outputs', exist_ok=True)
# note file saved in the outputs folder is automatically uploaded into experiment record
joblib.dump(value=model, filename='outputs/diabetes_model.pkl')

run.complete()

次に、データセットを推定器に渡す方法を変更する必要があります。スクリプトがファイルを読み取ることができるマウントポイントを定義する必要があります。大量のデータの場合、通常は**as_mount**メソッドを使用して、データセットソースから直接ファイルをストリーミングします。ただし、この例のようにローカルコンピューティングで実行する場合は、**as_download**オプションを使用して、データセットファイルをローカルフォルダーにダウンロードする必要があります。

また、**Dataset**クラスは**azureml-dataprep**パッケージで定義されているため、実験環境に含める必要があります。

In [ ]:
from azureml.train.sklearn import SKLearn
from azureml.core import Experiment
from azureml.widgets import RunDetails

# Set the script parameters
script_params = {
    '--regularization': 0.1
}

# Get the training dataset
diabetes_ds = ws.datasets.get("diabetes file dataset")

# Create an estimator
estimator = SKLearn(source_directory=experiment_folder,
                    entry_script='diabetes_training.py',
                    script_params=script_params,
                    compute_target = 'local',
                    inputs=[diabetes_ds.as_named_input('diabetes').as_download(path_on_compute='diabetes_data')], # Pass the Dataset object as an input
                    conda_packages=['pip==19.3.1'],
                    pip_packages=['azureml-dataprep[pandas]'] # so we need the dataprep package
                   )

# Create an experiment
experiment_name = 'diabetes-training'
experiment = Experiment(workspace = ws, name = experiment_name)

# Run the experiment
run = experiment.submit(config=estimator)
# Show the run details while running
RunDetails(run).show()
run.wait_for_completion()

実験が完了したら、ウィジェットで**azureml-logs/70_driver_log.txt**出力ログを表示して、ファイルデータセットが処理され、データファイルがダウンロードされたことを確認します。

すべての実験と同様に、[Azure ML Studio](https://ml.azure.com)で実験の詳細を表示でき、生成されたメトリックとファイルを取得するコードを作成できます。:

In [ ]:
# Get logged metrics
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')
for file in run.get_file_names():
    print(file)

もう一度、訓練したモデルを登録しましょう。

In [ ]:
from azureml.core import Model

run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                   tags={'Training context':'SKLearn Estimator (file dataset)'}, properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

> **詳しくは**: データセットを使用したトレーニングの詳細については、Azure MLドキュメントの[データセットを使用したトレーニング](https://docs.microsoft.com/azure/machine-learning/how-to-train-with-datasets)を参照してください。